In [1]:
import pandas as pd

In [2]:
rides = pd.read_parquet('../data/transformed/validated_rides_2022_01.parquet')
rides.head()

,pickup_datetime,pickup_location_id
0,2022-01-01 00:35:40,142
1,2022-01-01 00:33:43,236
2,2022-01-01 00:53:21,166
3,2022-01-01 00:25:21,114
4,2022-01-01 00:36:48,68


In [3]:
rides['pickup_hours'] = rides['pickup_datetime'].dt.floor('h')

In [4]:
rides

,pickup_datetime,pickup_location_id,pickup_hours
0,2022-01-01 00:35:40,142,2022-01-01 00:00:00
1,2022-01-01 00:33:43,236,2022-01-01 00:00:00
2,2022-01-01 00:53:21,166,2022-01-01 00:00:00
3,2022-01-01 00:25:21,114,2022-01-01 00:00:00
4,2022-01-01 00:36:48,68,2022-01-01 00:00:00
...,...,...,...
2463926,2022-01-31 23:36:53,90,2022-01-31 23:00:00
2463927,2022-01-31 23:44:22,107,2022-01-31 23:00:00
2463928,2022-01-31 23:39:00,113,2022-01-31 23:00:00
2463929,2022-01-31 23:36:42,148,2022-01-31 23:00:00


In [5]:
agg_rides = rides.groupby(['pickup_hours', 'pickup_location_id']).size().reset_index()
agg_rides.rename(columns={0: 'rides'}, inplace=True)
agg_rides

,pickup_hours,pickup_location_id,rides
0,2022-01-01 00:00:00,4,11
1,2022-01-01 00:00:00,7,6
2,2022-01-01 00:00:00,10,1
3,2022-01-01 00:00:00,12,2
4,2022-01-01 00:00:00,13,12
...,...,...,...
66863,2022-01-31 23:00:00,261,4
66864,2022-01-31 23:00:00,262,8
66865,2022-01-31 23:00:00,263,26
66866,2022-01-31 23:00:00,264,24


In [6]:
from tqdm import tqdm

def add_missing_slots(agg_rides: pd.DataFrame) -> pd.DataFrame:

    location_ids = agg_rides['pickup_location_id'].unique()
    full_range = pd.date_range(
        agg_rides['pickup_hours'].min(), agg_rides['pickup_hours'].max(), freq='h'
    )
    output = pd.DataFrame()
    for location_id in tqdm(location_ids):

        # keep only rides for this location_id
        agg_rides_i = agg_rides.loc[agg_rides['pickup_location_id'] == location_id, ['pickup_hours', 'rides']]

        # quick way to add missing dates with 0 in a series
        agg_rides_i.set_index('pickup_hours', inplace=True)
        agg_rides_i.index = pd.DatetimeIndex(agg_rides_i.index)
        agg_rides_i = agg_rides_i.reindex(full_range, fill_value=0)

        # add back location_id columns
        agg_rides_i['pickup_location_id'] = location_id

        # append to output
        output = pd.concat([output, agg_rides_i])

    # move the purchase_day from the index to a dataframe column
    output = output.reset_index().rename(columns={'index': 'pickup_hour'})
    
    return output

In [7]:
agg_rides_all_slots = add_missing_slots(agg_rides)

  0%|          | 0/257 [00:00<?, ?it/s]

100%|██████████| 257/257 [00:01<00:00, 134.25it/s]


In [8]:
from typing import Optional, List
import plotly.express as px

def plot_rides(
        rides: pd.DataFrame,
        locations: Optional[List[int]] = None):
    """
    Plot time-series data
    """
    rides_to_plot = rides[rides['pickup_location_id'].isin(locations)] if locations else rides

    fig = px.line(rides_to_plot,
                  x='pickup_hour',
                  y='rides',
                  color='pickup_location_id',
                  template='none',
                  )
    
    fig.show()

In [9]:
plot_rides(agg_rides_all_slots, locations=[43])

In [10]:
agg_rides_all_slots.to_parquet("../data/transformed/ts_data_2022_01.parquet")